# DWI preprocessing
## using MRTRIX, FSL and ANTS
### by Michael Paquette
#### Notes from Dec 15th

In this serie of bash notebook, I will be describing a basic DWI data preprocessing pipeline.

I will try to stick to this (loose) general format to describe each step:  

- What is the artefact we are trying to correct or the transformation we are trying to achieve  
- Why does this happen  
- What would happen if we didnt correct/modify 
- Why is this step here (somewhat arbitrary or specific ordering)  
- Important parameters of the tool (and which one are likely to need finetuning)  
- What to look for when doing QC (quality control)

In [1]:
# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'

# Like last time, we setup the basic variable with the folders
ROOTDIR='/data/pt_02586/'
SUBJECTDIR=$ROOTDIR'sub_01/'


: 1

In [2]:
# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'

# and we move to the preprocessing folder
cd $SUBJECTDIR'preprocessing'


: 1

In [3]:
pwd

pwd
/data/pt_02586/sub_01/preprocessing


: 1

In [4]:
# our dwi data is converted to nifti format,
# with strides 1,2,3,4
# and datatype float32
mrinfo $SUBJECTDIR'preprocessing/dwi.nii.gz'

# our dwi data is converted to nifti format,
# with strides 1,2,3,4
# and datatype float32
mrinfo $SUBJECTDIR'preprocessing/dwi.nii.gz'
************************************************
Image name:          "/data/pt_02586/sub_01/preprocessing/dwi.nii.gz"
************************************************
  Dimensions:        128 x 128 x 88 x 71
  Voxel size:        1.71875 x 1.71875 x 1.7 x 4.5
  Data strides:      [ 1 2 3 4 ]
  Format:            NIfTI-1.1 (GZip compressed)
  Data type:         32 bit float (little endian)
  Intensity scaling: offset = 0, multiplier = 1
  Transform:                    1           0          -0      -107.6
                                0           1          -0        -103
                               -0           0           1      -50.63
  comments:          TE=60;Time=135542
  mrtrix_version:    3.0.0


: 1

### Denoising

#### What are we correcting?
"Noise" in an MRI image is a deviation from the true signal value.   
#### Why does this happen?
random thermal fluctuation in the sensor.
#### What if we ignore it?
Noise makes regions less homogenous than they should be and leads to bias in model estimation.  
#### Why correct it now?
Correcting noise depend on having some prior knowledge of its mathematical form. For instance, any steps that involve registration (like motion correction or distorsion correction) will involve some amount of interpolation which will destroy our priors on the noise.  
#### correction tool: dwidenoise
dwidenoise is the MRtrix denoising (for dwi, it requires a 4th dimension of 30+ images).  
It is very convenient because it doesnt require to estimate the level of noise before, doesnt require masking and it doesnt blur the image.  
It is a PCA thresholding type of denoising (more later).  
In a nutshell, it takes small 5x5x5 blocks in space across all bvecs and exploit similarity + known properties of random noise.  
There are no relevant flags to change, default parameter are good. I am only using the -noise flag to output the estimated noise levels (standard deviations of a normal distribution in each 5x5x5 block)  
If you have a very large number of diffusion directions, you might need to use a bigger windows like 7x7x7 by adding the flag '-extent 7'. This is because you need more voxel in your windows than the amount of image you have in the 4th dimension. So for instance the default 5x5x5 kernel has 5^3 = 125 voxels, so its enough for the the 70 volumes we have.  
#### Quality control
- When looking at the noise that was removed, there should not be obvious anatomical structure apart from ventricule and outline of the brain/skull.  
- When looking at the map of sigma noise level, its should be relatively smooth with higher values in the center.  

#### Denoising can introduce negative values in the image, proper care (like clipping them to 0) will be required later

In [14]:
dwidenoise $SUBJECTDIR'preprocessing/dwi.nii.gz' \
           $SUBJECTDIR'preprocessing/dwi_denoise.nii.gz' \
           -noise $SUBJECTDIR'preprocessing/mppca_sigma.nii.gz'

# dwidenoise $SUBJECTDIR'preprocessing/dwi.nii.gz' \
#            $SUBJECTDIR'preprocessing/dwi_denoise.nii.gz' \
'            -noise $SUBJECTDIR'preprocessing/mppca_sigma.nii.gz'


: 1

In [19]:
# Compute a map of the removed noise with "mrcalc"
# residual = noisy minus denoised
mrcalc $SUBJECTDIR'preprocessing/dwi.nii.gz' \
       $SUBJECTDIR'preprocessing/dwi_denoise.nii.gz' \
       -subtract $SUBJECTDIR'preprocessing/dwi_denoise_residual.nii.gz'

# Compute a map of the removed noise with "mrcalc"
# residual = noisy minus denoised
# mrcalc $SUBJECTDIR'preprocessing/dwi.nii.gz' \
#        $SUBJECTDIR'preprocessing/dwi_denoise.nii.gz' \
l.nii.gz'-subtract $SUBJECTDIR'preprocessing/dwi_denoise_residual


: 1

In [21]:
# We compute the absolute value of the residual with "mrabs"
# This is useful because the removed noise can be + or -
# but we mostly care about the "amount" of it that was removed
mrabs $SUBJECTDIR'preprocessing/dwi_denoise_residual.nii.gz' \
      $SUBJECTDIR'preprocessing/dwi_denoise_ABSresidual.nii.gz'

# We compute the absolute value of the residual with "mrabs"
# This is useful because the removed noise can be + or -
# but we mostly care about the "amount" of it that was removed
mrabs $SUBJECTDIR'preprocessing/dwi_denoise_residual.nii.gz' \
      $SUBJECTDIR'preprocessing/dwi_denoise_ABSresidual.nii.gz'
mrabs: taking absolute value... 100%
mrabs: error creating file "/data/pt_02586/sub_01/preprocessing/dwi_denoise_ABSresidual.nii.gz": File exists
terminate called after throwing an instance of 'MR::Exception'
Aborted


: 1

### MRtrix viewer: mrview
mrview is a good viewer for QC (better then fsleyes) because its really simple and fast to switch between images (pageUp and pageDown keyboard keys cycle through all the opened images).  
Typical QC use-case:  
mrview image_before.nii.gz image_after.nii.gz  
Other useful keyboard shortcut:  
i: disable/enable image (i)nterpolation  
esc: auto adjust contract with respect to current 2D image  
a/s/c: change the view axis between (a)xial (s)aggital and (c)oronal.  
left and right arrow: move in the 4th dimension of the image.

In [23]:
mrview $SUBJECTDIR'preprocessing/dwi_denoise_ABSresidual.nii.gz'

'


: 1

### Gibbs ringing correction

#### What are we correcting?
We are correcting the gibbs ringing artefact.  
#### Why does this happen?
Since MRI acquisition is based on the (finite) fourier space of the image, sharp edges are misrepresented.  
#### What if we ignore it?
Gibbs ringing looks like noise that follow the shape of edges and fade as you get away from them, it will bias estimation near edges.  
#### Why correct it now?
Since ringing has a very specific shape, you need to corect it before the image is interpolated.  
#### correction tool: mrdegibbs (2D version)
mrdegibbs is the gibbs ringing correction tool from the MRtrix software.  
We are using the default one which is a 2D version of the tool. It works one slice at the time and it is IMPORTANT to specify which axes of the image where the true slice orientation from the acquisition. Since gibbs ringing is a very specific acquisition related artefact, It will NOT work in the wrong orientation. The slice encoding orientation is encoded in the DICOM header, when we ran "mrinfo" on the DICOM previously, it showed "SliceEncodingDirection: k" in the extra information, this mean that the slice encoding direction is the 3rd axe (it goes (i, j, k)), therefore we will run the tool on every (i, j) 2D slice.  
The tool has a -axes flag to specify this, "-axes 0,1" in this case (because they start counting at 0).
#### Quality control
- When looking at the image of the removed ringing (residual), it should look like the edges of the brain, the skull, the ventricule and other features with large intensity change with oscilliating "ghost" edges fading away as they get more distant from the true edge.
#### Degibbsing can introduce negative values in the image, proper care (like clipping them to 0) will be required later


In [24]:
mrdegibbs $SUBJECTDIR'preprocessing/dwi_denoise.nii.gz' \
          $SUBJECTDIR'preprocessing/dwi_denoise_degibbs.nii.gz' \
          -axes 0,1

# mrdegibbs $SUBJECTDIR'preprocessing/dwi_denoise.nii.gz' \
' \         $SUBJECTDIR'preprocessing/dwi_denoise_degibbs.nii.gz'
#           -axes 0,1


: 1

In [ ]:
# As before, we compute residuals
mrcalc $SUBJECTDIR'preprocessing/dwi_denoise.nii.gz' \
       $SUBJECTDIR'preprocessing/dwi_denoise_degibbs.nii.gz' \
       -subtract $SUBJECTDIR'preprocessing/dwi_denoise_degibbs_residual.nii.gz'

In [ ]:
# We also compute absolute residual (which are less usuful for degibbs)
mrabs $SUBJECTDIR'preprocessing/dwi_denoise_degibbs_residual.nii.gz' \
      $SUBJECTDIR'preprocessing/dwi_denoise_degibbs_ABSresidual.nii.gz'